In [2]:
import copy
import time

def tree_search(problem, strategy):
    source_node = Node(problem, None, None, None)
    open_list = [source_node]
    closed_list = []

    start_time = time.perf_counter()

    while open_list:
        current_node = strategy.get_next(open_list)
        closed_list.append(current_node)
        if current_node.value.is_valid_result():
            end_time = time.perf_counter()

            return SearchResult(True, end_time - start_time, closed_list)
        strategy.expand(current_node, open_list)

    end_time = time.time_ns()

    return SearchResult(False, end_time - start_time, closed_list)


class BFS:
    def expand(self, node, open_list):
        """
        Expands the Problem within the given node and expands them according to the strategy
        :param node: the current node to be expanded
        :param open_list: the open list, where new nodes are added
        :return: None
        """
        new_values = node.value.move()
        for v in new_values:
            created_node = Node(Problem(v), node, 0, None)  # create new node for the openlist
            node.add_children(created_node)  # add the new node as children to the parent node
            open_list.append(created_node)  # add the new node to the openlist

    def get_next(self, open_list):
        """
        Returns the next node to be expanded, according to the strategy
        :param open_list: the open list with all open nodes
        :return: the next node to be expanded
        """
        return open_list.pop(0)


class DFS:
    def expand(self, node, open_list):
        """
        Expands the Problem within the given node and expands them according to the strategy
        :param node: the current node to be expanded
        :param open_list: the open list, where new nodes are added
        :return: None
        """
        new_values = node.value.move()
        for v in new_values:
            created_node = Node(Problem(v), node, 0, None)  # create new node for the openlist
            node.add_children(created_node)  # add the new node as children to the parent node
            open_list.append(created_node)  # add the new node to the openlist

    def get_next(self, open_list):
        """
        Returns the next node to be expanded, according to the strategy
        :param open_list: the open list with all open nodes
        :return: the next node to be expanded
        """
        return open_list.pop()


class IDS:
    def __init__(self, max_depth):
        self.max_depth = max_depth

    def expand(self, node, open_list):
        """
        Expands the Problem within the given node and expands them according to the strategy
        :param node: the current node to be expanded
        :param open_list: the open list, where new nodes are added
        :return: None
        """
        if node.depth < self.max_depth:
            new_values = node.value.move()
            for v in new_values:
                created_node = Node(Problem(v), node, 0, None)  # create new node for the openlist
                node.add_children(created_node)  # add the new node as children to the parent node
                open_list.append(created_node)  # add the new node to the openlist

    def get_next(self, open_list):
        """
        Returns the next node to be expanded, according to the strategy
        :param open_list: the open list with all open nodes
        :return: the next node to be expanded
        """
        return open_list.pop()


class AStar:
    def expand(self, node, open_list):
        """
        Expands the Problem within the given node and expands them according to the strategy
        :param node: the current node to be expanded
        :param open_list: the open list, where new nodes are added
        :return: None
        """
        new_values = node.value.move()
        for v in new_values:
            new_problem = Problem(v)
            expected_rest_cost = self.heuristic_idea(new_problem) # 0
            total_cost = expected_rest_cost + node.depth + 1
            created_node = Node(new_problem, node, total_cost, None)  # create new node for the openlist
            node.add_children(created_node)  # add the new node as children to the parent node

            i = 0

            while i < len(open_list) or len(open_list) == 0:
                if len(open_list) == 0 or open_list[i].weight < total_cost or i == len(open_list) - 1:
                    open_list.insert(i, created_node)  # add the new node to the openlist
                    i = len(open_list) + 1
                else:
                    i += 1

    def get_next(self, open_list):
        """
        Returns the next node to be expanded, according to the strategy
        :param open_list: the open list with all open nodes
        :return: the next node to be expanded
        """
        return open_list.pop()

    def heuristic_idea(self, problem):
        return 0
        # return 15 - problem.get_curr_path_length()


class SearchResult:
    def __init__(self, success, runtime, solution):
        self.success: bool = success
        self.runtime = runtime
        self.solution: list = solution

    def __str__(self):
        return (f"------------\n"
                f"Solution found? {self.success}\n"
                f"Runtime: {self.runtime}\n"
                f"Visited nodes: {len(self.solution)}\n"
                f"Final matrix: {self.solution[-1].value.matrix}")


class Node:
    def __init__(self, value, parent, weight, children):
        """
        Initialises the new Node
        :param value: the value stored inside the node
        :param parent: reference to the parent node
        :param weight: the weight of the node
        :param children: the child nodes of this node
        """
        self.value = value
        self.parent = parent
        self.weight = weight
        self.children = children if children is not None else []
        self.depth = self.parent.depth + 1 if self.parent is not None else 0

    def add_children(self, node):
        """Ads the given node as a child
        :param node: the node to be added
        """
        self.children.append(node)


class Problem:
    def __init__(self, start_matrix):
        """
        :param start_matrix: the matrix as an array [[][]]
        """
        self.matrix: list = start_matrix
        self.empty_pos: list = self.find_pos(0)
        self.start_pos: list = self.find_pos(1)

    def move(self):
        """
        Moves the blank space through the problem array in all possible direction
        :return: a list over all newly created arrays
        """
        empty_pos_x: int = self.empty_pos[0]
        empty_pos_y: int = self.empty_pos[1]
        new_matrices: list = []

        if empty_pos_x - 1 >= 0:
            new_matrix = copy.deepcopy(self.matrix)
            new_matrix[empty_pos_x][empty_pos_y] = new_matrix[empty_pos_x - 1][empty_pos_y]
            new_matrix[empty_pos_x - 1][empty_pos_y] = 0
            new_matrices.append(new_matrix)
        if empty_pos_y - 1 >= 0:
            new_matrix = copy.deepcopy(self.matrix)
            new_matrix[empty_pos_x][empty_pos_y] = new_matrix[empty_pos_x][empty_pos_y - 1]
            new_matrix[empty_pos_x][empty_pos_y - 1] = 0
            new_matrices.append(new_matrix)
        if empty_pos_x + 1 <= len(self.matrix) - 1:
            new_matrix = copy.deepcopy(self.matrix)
            new_matrix[empty_pos_x][empty_pos_y] = new_matrix[empty_pos_x + 1][empty_pos_y]
            new_matrix[empty_pos_x + 1][empty_pos_y] = 0
            new_matrices.append(new_matrix)
        if empty_pos_y + 1 <= len(self.matrix[empty_pos_x]) - 1:
            new_matrix = copy.deepcopy(self.matrix)
            new_matrix[empty_pos_x][empty_pos_y] = new_matrix[empty_pos_x][empty_pos_y + 1]
            new_matrix[empty_pos_x][empty_pos_y + 1] = 0
            new_matrices.append(new_matrix)

        return new_matrices

    def find_pos(self, target):
        """
        Finds and returns the position of any given number within the problem array
        :param target: the number to be found
        :return: an array with the coordinates of the number: [x , y]
        :raises NotInMatrixException
        """
        for i in range(0, len(self.matrix)):
            for j in range(0, len(self.matrix[i])):
                if target == self.matrix[i][j]:
                    return [i, j]
        raise NotInMatrixException

    def is_valid_result(self):
        """
        Checks if the current configuration of the array is a valid solution for the problem: i.e. it is possible
        for the chess knight to go from one to 15 within the array
        :return: true if the configuration is valid, false if not
        """
        reachable = True  # path with only one always exists
        next_target = 2  # starting from one the number two is the next to be reached
        current_pos = self.start_pos  # pos of the one in the array

        while reachable and next_target <= 15:
            reachable = self.is_reachable(current_pos, next_target)  # check if path is continuing
            current_pos = self.find_pos(next_target)  # set current position to the right number
            next_target += 1  # set next target

        return reachable

    def get_curr_path_length(self):
        """
        Returns the current max path length for the chess knight starting from the number one
        :return: the number of possible steps
        """
        next_target = 2
        current_pos = self.start_pos
        reachable = True
        while reachable:
            reachable = self.is_reachable(current_pos, next_target)
            if not reachable:
                return next_target - 1
            current_pos = self.find_pos(next_target)
            next_target += 1
        return next_target - 1  # evtl. exception

    def is_reachable(self, source_pos, target_value):
        """
        Checks if the given number is reachable from the defined source, according to the
        possible movements of the chess knight

        :param source_pos: the starting point in the problem array, pos 0 is the source_pos_x coordinate and pos 1 the source_pos_y coordinate
        :param target_value: the number to be reached from the source point
        :return: true if the point is reachable, false if not
        """
        source_pos_x = source_pos[0]
        source_pos_y = source_pos[1]

        if source_pos_x - 1 >= 0 and source_pos_y + 2 < len(self.matrix) and self.matrix[source_pos_x - 1][source_pos_y + 2] == target_value:
            return True
        elif source_pos_x - 1 >= 0 and source_pos_y - 2 >= 0 and self.matrix[source_pos_x - 1][source_pos_y - 2] == target_value:
            return True
        elif source_pos_x + 1 < len(self.matrix) and source_pos_y + 2 < len(self.matrix) and self.matrix[source_pos_x + 1][source_pos_y + 2] == target_value:
            return True
        elif source_pos_x + 1 < len(self.matrix) and source_pos_y - 2 >= 0 and self.matrix[source_pos_x + 1][source_pos_y - 2] == target_value:
            return True
        elif source_pos_x - 2 >= 0 and source_pos_y + 1 < len(self.matrix) and self.matrix[source_pos_x - 2][source_pos_y + 1] == target_value:
            return True
        elif source_pos_x - 2 >= 0 and source_pos_y - 1 >= 0 and self.matrix[source_pos_x - 2][source_pos_y - 1] == target_value:
            return True
        elif source_pos_x + 2 < len(self.matrix) and source_pos_y + 1 < len(self.matrix) and self.matrix[source_pos_x + 2][source_pos_y + 1] == target_value:
            return True
        elif source_pos_x + 2 < len(self.matrix) and source_pos_y - 1 >= 0 and self.matrix[source_pos_x + 2][source_pos_y - 1] == target_value:
            return True
        else:
            return False


class NotInMatrixException(BaseException):
    pass


example_1 = [[15, 10, 3, 6], [4, 7, 14, 11], [0, 12, 5, 2], [9, 1, 8, 13]]
example_2 = [[15, 10, 3, 6], [4, 7, 14, 11], [12, 0, 5, 2], [9, 1, 8, 13]]
example_3 = [[15, 10, 3, 6], [4, 14, 0, 11], [12, 7, 5, 2], [9, 1, 8, 13]]

print("- BFS ------")
print(tree_search(Problem(example_1), BFS()))
print(tree_search(Problem(example_2), BFS()))
print(tree_search(Problem(example_3), BFS()))

print("- IDS ------")
print(tree_search(Problem(example_1), IDS(4)))
print(tree_search(Problem(example_2), IDS(4)))
print(tree_search(Problem(example_3), IDS(4)))

print("- AStar ------")
print(tree_search(Problem(example_1), AStar()))
print(tree_search(Problem(example_2), AStar()))
print(tree_search(Problem(example_3), AStar()))

# print("- DFS ------")
# print(tree_search(Problem(example_1), DFS()))
# print(tree_search(Problem(example_2), DFS()))
# print(tree_search(Problem(example_3), DFS()))

- BFS ------
------------
Solution found? True
Runtime: 0.00200749997748062
Visited nodes: 3
Final matrix: [[15, 10, 3, 6], [4, 7, 14, 11], [9, 12, 5, 2], [0, 1, 8, 13]]
------------
Solution found? True
Runtime: 0.007081624993588775
Visited nodes: 11
Final matrix: [[15, 10, 3, 6], [4, 7, 14, 11], [9, 12, 5, 2], [0, 1, 8, 13]]
------------
Solution found? True
Runtime: 0.04880625003715977
Visited nodes: 118
Final matrix: [[15, 10, 3, 6], [4, 7, 14, 11], [9, 12, 5, 2], [0, 1, 8, 13]]
- IDS ------
------------
Solution found? True
Runtime: 0.00951570796314627
Visited nodes: 41
Final matrix: [[15, 10, 3, 6], [4, 7, 14, 11], [9, 12, 5, 2], [0, 1, 8, 13]]
------------
Solution found? True
Runtime: 0.009016374999191612
Visited nodes: 41
Final matrix: [[15, 10, 3, 6], [4, 7, 14, 11], [9, 12, 5, 2], [0, 1, 8, 13]]
------------
Solution found? True
Runtime: 0.017865790985524654
Visited nodes: 141
Final matrix: [[15, 10, 3, 6], [4, 7, 14, 11], [9, 12, 5, 2], [0, 1, 8, 13]]
- AStar ------
-------